<a href="https://colab.research.google.com/github/Sahilgupta100/pw_skills_assignment/blob/main/Robotics_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Coordinate Transformation And Its Inverse

In [1]:
import numpy as np

def translation_matrix(tx, ty, tz):
    """Returns a 4x4 translation matrix."""
    return np.array([
        [1, 0, 0, tx],
        [0, 1, 0, ty],
        [0, 0, 1, tz],
        [0, 0, 0, 1]
    ])

def rotation_matrix_x(theta):
    """Returns a 4x4 rotation matrix around the X-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [1, 0,  0, 0],
        [0, c, -s, 0],
        [0, s,  c, 0],
        [0, 0,  0, 1]
    ])

def rotation_matrix_y(theta):
    """Returns a 4x4 rotation matrix around the Y-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [ c, 0, s, 0],
        [ 0, 1, 0, 0],
        [-s, 0, c, 0],
        [ 0, 0, 0, 1]
    ])

def rotation_matrix_z(theta):
    """Returns a 4x4 rotation matrix around the Z-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [c, -s, 0, 0],
        [s,  c, 0, 0],
        [0,  0, 1, 0],
        [0,  0, 0, 1]
    ])

def homogeneous_transformation(tx, ty, tz, rx, ry, rz):
    """Returns a homogeneous transformation matrix given translation and rotation angles."""
    T = translation_matrix(tx, ty, tz) @ rotation_matrix_x(rx) @ rotation_matrix_y(ry) @ rotation_matrix_z(rz)
    return T

def inverse_transformation(T):
    """Returns the inverse of a given homogeneous transformation matrix."""
    R = T[:3, :3]
    t = T[:3, 3]
    T_inv = np.eye(4)
    T_inv[:3, :3] = R.T
    T_inv[:3, 3] = -R.T @ t
    return T_inv

# Example usage
if __name__ == "__main__":
    tx, ty, tz = 1, 2, 3  # Translation values
    rx, ry, rz = np.radians(30), np.radians(45), np.radians(60)  # Rotation angles in radians

    T = homogeneous_transformation(tx, ty, tz, rx, ry, rz)
    print("Homogeneous Transformation Matrix:")
    print(T)

    T_inv = inverse_transformation(T)
    print("\nInverse Transformation Matrix:")
    print(T_inv)


Homogeneous Transformation Matrix:
[[ 0.35355339 -0.61237244  0.70710678  1.        ]
 [ 0.9267767   0.12682648 -0.35355339  2.        ]
 [ 0.12682648  0.78033009  0.61237244  3.        ]
 [ 0.          0.          0.          1.        ]]

Inverse Transformation Matrix:
[[ 0.35355339  0.9267767   0.12682648 -2.58758623]
 [-0.61237244  0.12682648  0.78033009 -1.98227079]
 [ 0.70710678 -0.35355339  0.61237244 -1.83711731]
 [ 0.          0.          0.          1.        ]]


Rotation About General Axis

In [2]:
import numpy as np

def rotation_matrix(axis, angle):
    """
    Compute the rotation matrix for a rotation around an arbitrary axis.

    :param axis: 3-element list or array representing the axis of rotation (must be a unit vector)
    :param angle: Rotation angle in radians
    :return: 3x3 rotation matrix
    """
    axis = np.array(axis, dtype=float)
    axis = axis / np.linalg.norm(axis)  # Normalize to make it a unit vector

    kx, ky, kz = axis
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)

    # Skew-symmetric matrix K
    K = np.array([[0, -kz, ky],
                  [kz, 0, -kx],
                  [-ky, kx, 0]])

    # Compute rotation matrix using Rodrigues' formula
    R = np.eye(3) + sin_theta * K + (1 - cos_theta) * np.dot(K, K)
    return R

def rotate_point(point, axis, angle):
    """
    Rotate a 3D point around an arbitrary axis.

    :param point: 3-element list or array representing the 3D point
    :param axis: 3-element list or array representing the axis of rotation (must be a unit vector)
    :param angle: Rotation angle in radians
    :return: Rotated 3D point as a numpy array
    """
    R = rotation_matrix(axis, angle)
    rotated_point = np.dot(R, point)
    return rotated_point

# Example Usage
if __name__ == "__main__":
    axis = [1, 1, 1]  # General axis (will be normalized)
    angle = np.pi / 4  # Rotate by 45 degrees
    point = [1, 0, 0]  # Initial point

    rotated_point = rotate_point(point, axis, angle)
    print("Rotated Point:", rotated_point)


Rotated Point: [ 0.80473785  0.50587936 -0.31061722]


Homogeneous Transformation

In [3]:
import numpy as np

class HomogeneousTransformation:
    def __init__(self, rotation_matrix, translation_vector):
        """
        Initialize the homogeneous transformation matrix.
        :param rotation_matrix: 3x3 numpy array representing rotation
        :param translation_vector: 3x1 numpy array representing translation
        """
        self.T = np.eye(4)  # Initialize 4x4 identity matrix
        self.T[:3, :3] = rotation_matrix  # Set rotation part
        self.T[:3, 3] = translation_vector.flatten()  # Set translation part

    def transform_point(self, point):
        """
        Transform a 3D point using the homogeneous transformation matrix.
        :param point: 3x1 numpy array representing the point
        :return: Transformed 3x1 numpy array
        """
        point_h = np.append(point, 1)  # Convert to homogeneous coordinates
        transformed_point_h = np.dot(self.T, point_h)  # Apply transformation
        return transformed_point_h[:3]  # Convert back to 3D coordinates

    def inverse_transform(self):
        """
        Compute the inverse of the transformation.
        :return: Inverse transformation as a new HomogeneousTransformation object
        """
        R_inv = self.T[:3, :3].T  # Transpose of the rotation matrix
        t_inv = -np.dot(R_inv, self.T[:3, 3])  # Compute inverse translation
        return HomogeneousTransformation(R_inv, t_inv)

# Example Usage
if __name__ == "__main__":
    R = np.array([[0, -1, 0],  # 90-degree rotation about Z-axis
                  [1, 0, 0],
                  [0, 0, 1]])
    t = np.array([2, 3, 1])  # Translation vector

    transform = HomogeneousTransformation(R, t)

    point = np.array([1, 1, 0])  # Example point in original frame
    transformed_point = transform.transform_point(point)
    print("Transformed Point:", transformed_point)

    inverse_transform = transform.inverse_transform()
    original_point = inverse_transform.transform_point(transformed_point)
    print("Recovered Original Point:", original_point)


Transformed Point: [1. 4. 1.]
Recovered Original Point: [1. 1. 0.]


Velocity of a point

In [4]:
import numpy as np

def velocity_of_point(V_O, omega, r):
    """
    Compute the velocity of a point in a moving frame.

    :param V_O: 3x1 numpy array representing the linear velocity of the origin
    :param omega: 3x1 numpy array representing the angular velocity vector
    :param r: 3x1 numpy array representing the position vector of the point relative to the origin
    :return: 3x1 numpy array representing the velocity of the point
    """
    V_P = V_O + np.cross(omega, r)  # Compute velocity using cross-product
    return V_P

# Example Usage
if __name__ == "__main__":
    V_O = np.array([1, 2, 0])  # Linear velocity of the moving frame origin
    omega = np.array([0, 0, 2])  # Angular velocity (about Z-axis)
    r = np.array([3, 4, 0])  # Position of the point relative to the moving frame origin

    V_P = velocity_of_point(V_O, omega, r)
    print("Velocity of the point P:", V_P)


Velocity of the point P: [-7  8  0]


Arm Matrix

In [5]:
import numpy as np

def dh_transform_matrix(theta, d, a, alpha):
    """
    Compute the Denavit-Hartenberg (D-H) transformation matrix.

    :param theta: Joint angle (in radians)
    :param d: Link offset
    :param a: Link length
    :param alpha: Link twist (in radians)
    :return: 4x4 Transformation Matrix
    """
    ct, st = np.cos(theta), np.sin(theta)
    ca, sa = np.cos(alpha), np.sin(alpha)

    return np.array([
        [ct, -st * ca,  st * sa, a * ct],
        [st,  ct * ca, -ct * sa, a * st],
        [0,       sa,      ca,       d],
        [0,        0,       0,       1]
    ])

def forward_kinematics(dh_params):
    """
    Compute the forward kinematics using a list of D-H parameters.

    :param dh_params: List of tuples (theta, d, a, alpha) for each joint
    :return: 4x4 Homogeneous Transformation Matrix of End-Effector
    """
    T = np.eye(4)  # Identity matrix (Base Frame)
    for params in dh_params:
        T_i = dh_transform_matrix(*params)  # Compute transformation for this joint
        T = np.dot(T, T_i)  # Multiply transformation matrices
    return T

# Example Usage (2-link robotic arm)
if __name__ == "__main__":
    # Example D-H parameters (theta in radians, d, a, alpha in radians)
    dh_params = [
        (np.radians(30),  2,  3, np.radians(0)),  # Joint 1
        (np.radians(45),  0,  2, np.radians(0))   # Joint 2
    ]

    T_end_effector = forward_kinematics(dh_params)
    print("End-Effector Transformation Matrix:\n", T_end_effector)


End-Effector Transformation Matrix:
 [[ 0.25881905 -0.96592583  0.          3.1157143 ]
 [ 0.96592583  0.25881905  0.          3.43185165]
 [ 0.          0.          1.          2.        ]
 [ 0.          0.          0.          1.        ]]
